# Importing

In [2]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [3]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)

In [4]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [5]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 16813124 (64.14 MB)
Trainable params: 16813124 (64.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
conv_base.trainable = False

In [8]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory='../BrainTumor/Training',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    # shuffle=True,
    # seed=42,
    # subset='training'  # Use this subset for training
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='../BrainTumor/Testing',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    # shuffle=True,
    # seed=42,
    # subset='validation'  # Use this subset for validation
)

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.


In [9]:
# Normalize
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [11]:
history = model.fit(train_ds,epochs=4,validation_data=validation_ds)

Epoch 1/4


90/90 [==============================] - 189s 2s/step - loss: 0.8194 - accuracy: 0.7157 - val_loss: 1.3888 - val_accuracy: 0.6091
Epoch 2/4
90/90 [==============================] - 185s 2s/step - loss: 0.3555 - accuracy: 0.8676 - val_loss: 1.4248 - val_accuracy: 0.6320
Epoch 3/4
90/90 [==============================] - 182s 2s/step - loss: 0.2498 - accuracy: 0.9115 - val_loss: 1.4397 - val_accuracy: 0.6421
Epoch 4/4
90/90 [==============================] - 179s 2s/step - loss: 0.1975 - accuracy: 0.9321 - val_loss: 1.5049 - val_accuracy: 0.7208
